In [1]:
import pandas as pd
import sqlite3

In [2]:
pd.options.display.max_columns = None

In [3]:
conn = sqlite3.connect('sakila.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
SELECT * FROM sqlite_master WHERE type = 'table'
''')
data = cursor.fetchall()
columns = [col[0] for col in cursor.description]
cursor.close()

In [5]:
data_df = pd.DataFrame(data, columns=columns)
data_df

,type,name,tbl_name,rootpage,sql
0,table,actor,actor,2,"CREATE TABLE ""actor"" (\n""index"" INTEGER,\n ""a..."
1,table,store,store,7,"CREATE TABLE ""store"" (\n""index"" INTEGER,\n ""s..."
2,table,address,address,9,"CREATE TABLE ""address"" (\n""index"" INTEGER,\n ..."
3,table,category,category,26,"CREATE TABLE ""category"" (\n""index"" INTEGER,\n ..."
4,table,city,city,28,"CREATE TABLE ""city"" (\n""index"" INTEGER,\n ""ci..."
5,table,country,country,39,"CREATE TABLE ""country"" (\n""index"" INTEGER,\n ..."
6,table,customer,customer,43,"CREATE TABLE ""customer"" (\n""index"" INTEGER,\n ..."
7,table,film_actor,film_actor,63,"CREATE TABLE ""film_actor"" (\n""index"" INTEGER,\..."
8,table,film_category,film_category,126,"CREATE TABLE ""film_category"" (\n""index"" INTEGE..."
9,table,inventory,inventory,140,"CREATE TABLE ""inventory"" (\n""index"" INTEGER,\n..."


In [6]:
pd.read_sql_query(sql = '''
SELECT * FROM actor;
''', con = conn)

,index,actor_id,first_name,last_name,last_update
0,0,1,Penelope,Guiness,2013-05-26 14:47:57.620000
1,1,2,Nick,Wahlberg,2013-05-26 14:47:57.620000
2,2,3,Ed,Chase,2013-05-26 14:47:57.620000
3,3,4,Jennifer,Davis,2013-05-26 14:47:57.620000
4,4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620000
...,...,...,...,...,...
195,195,196,Bela,Walken,2013-05-26 14:47:57.620000
196,196,197,Reese,West,2013-05-26 14:47:57.620000
197,197,198,Mary,Keitel,2013-05-26 14:47:57.620000
198,198,199,Julia,Fawcett,2013-05-26 14:47:57.620000


Question 1:

Question 1:
We want to find out how the two stores compare in their count of rental orders during every month for all the years we have data for. Write a query that returns the store ID for the store, the year and month and the number of rental orders each store has fulfilled for that month. Your table should include a column for each of the following: year, month, store ID and count of rental orders fulfilled during that month.

Check Your Solution
The following table header provides a preview of what your table should look like. The count of rental orders is sorted in descending order.

HINT: One way to solve this query is the use of aggregations.

![title](question-set2-ques1.png)

In [7]:
# sqlite version, use solution for project

pd.read_sql_query(sql = '''
SELECT strftime('%Y', r.rental_date) AS year, strftime('%m', r.rental_date) AS month,
sta.store_id AS store_id, COUNT(r.rental_id) AS rentals_count
FROM rental r
JOIN staff sta
ON r.staff_id = sta.staff_id
JOIN store sto
ON sta.store_id = sto.store_id
GROUP BY 1, 2, 3
ORDER BY 4 DESC;
''', con = conn)

,year,month,store_id,rentals_count
0,2005,07,2,3367
1,2005,07,1,3342
2,2005,08,1,2892
3,2005,08,2,2794
4,2005,06,1,1163
5,2005,06,2,1148
6,2005,05,2,598
7,2005,05,1,558
8,2006,02,2,97
9,2006,02,1,85


Solution:

SELECT DATE_PART('year', r.rental_date) AS year, DATE_PART('month', r.rental_date) AS month,
sta.store_id AS store_id, COUNT(r.rental_id) AS rentals_count
FROM rental r
JOIN staff sta
ON r.staff_id = sta.staff_id
GROUP BY 1, 2, 3
ORDER BY 4 DESC;

In [14]:
# Check: Count of rentals = count of inventory!!!

pd.read_sql_query(sql = '''
SELECT strftime('%Y', r.rental_date) AS year, strftime('%m', r.rental_date) AS month,
sta.store_id AS store_id, COUNT(r.rental_id) AS rentals_count
FROM rental r
JOIN inventory AS i
ON r.inventory_id = i.inventory_id
JOIN staff sta
ON r.staff_id = sta.staff_id
JOIN store sto
ON sta.store_id = sto.store_id
GROUP BY 1, 2, 3
ORDER BY 4 DESC;
''', con = conn)

,year,month,store_id,rentals_count
0,2005,07,2,3367
1,2005,07,1,3342
2,2005,08,1,2892
3,2005,08,2,2794
4,2005,06,1,1163
5,2005,06,2,1148
6,2005,05,2,598
7,2005,05,1,558
8,2006,02,2,97
9,2006,02,1,85
